LIBRARIES

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
import glob
import cv2
import matplotlib.pyplot as plt
import os
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DATASET

In [ ]:
!wget -P 'content/drive/MyDrive/Deepfake/Dataset' https://iplab.dmi.unict.it/deepfakechallenge/training/0-CelebA.zip
!wget -P 'content/drive/MyDrive/Deepfake/Dataset' https://iplab.dmi.unict.it/deepfakechallenge/training/0-FFHQ.zip
!wget -P 'content/drive/MyDrive/Deepfake/Dataset' https://iplab.dmi.unict.it/deepfakechallenge/training/1-ATTGAN.zip
!wget -P 'content/drive/MyDrive/Deepfake/Dataset' https://iplab.dmi.unict.it/deepfakechallenge/portion/1-STARGAN.zip
!wget -P 'content/drive/MyDrive/Deepfake/Dataset' https://iplab.dmi.unict.it/deepfakechallenge/portion/1-STYLEGAN.zip
!wget -P 'content/drive/MyDrive/Deepfake/Dataset' https://iplab.dmi.unict.it/deepfakechallenge/portion/1-STYLEGAN2.zip
!wget -P 'content/drive/MyDrive/Deepfake/Dataset' https://iplab.dmi.unict.it/deepfakechallenge/test/test-task1.zip

In [ ]:
!rm -r '/content/sample_data/1-StarGAN' '/content/sample_data/0-CelebA' '/content/sample_data/0-FFHQ' '/content/sample_data/1-STYLEGAN' '/content/sample_data/1-STYLEGAN2' '/content/sample_data/test-task1'

In [ ]:
!unzip -uq "/content/drive/MyDrive/Deepfake/Dataset/0-CelebA.zip" -d "/content/sample_data/"
!unzip -uq "/content/drive/MyDrive/Deepfake/Dataset/0-FFHQ.zip" -d "/content/sample_data/"

In [ ]:
!unzip -uq "/content/drive/MyDrive/Deepfake/Dataset/1-ATTGAN.zip" -d "/content/sample_data/"
!unzip -uq "/content/drive/MyDrive/Deepfake/Dataset/1-STARGAN.zip" -d "/content/sample_data/"
!unzip -uq "/content/drive/MyDrive/Deepfake/Dataset/1-STYLEGAN2.zip" -d "/content/sample_data/"
!unzip -uq "/content/drive/MyDrive/Deepfake/Dataset/1-STYLEGAN.zip" -d "/content/sample_data/"


In [ ]:
!unzip -uq "/content/drive/MyDrive/Deepfake/Dataset/test-task1.zip" -d "/content/sample_data/"

In [ ]:
!mkdir '/content/sample_data/train'
!mv '/content/sample_data/0-CelebA' '/content/sample_data/train/real'
!mv '/content/sample_data/0-FFHQ' '/content/sample_data/train/real'
!mv '/content/sample_data/1-STARGAN' '/content/sample_data/train/fake' 
!mv '/content/sample_data/1-STYLEGAN' '/content/sample_data/train/fake' 
!mv '/content/sample_data/1-STYLEGAN2' '/content/sample_data/train/fake' 
!mv '/content/sample_data/1-ATTGAN' '/content/sample_data/train/fake' 

In [ ]:
# moving test data to 2 folders for labelling
!wget "/content/sample_data/" https://iplab.dmi.unict.it/deepfakechallenge/test/label-task1.txt



In [ ]:
Labels = []

f = open('/content/label-task1.txt')
for line in f:
  Labels.append(int(line[-2]))


CODE IMPLEMENTATION

In [ ]:

ds = tf.keras.utils.image_dataset_from_directory(
    '/content/sample_data/train',
    labels='inferred',
    label_mode='binary',
    class_names=None,
    color_mode='rgb',
    batch_size=128,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)

total = len(ds)

train_ds = ds.take(int(total*0.8))
val_ds = ds.skip(int(total*0.2))


In [ ]:
IMG_SIZE = 256

resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.Resizing(IMG_SIZE, IMG_SIZE),
  tf.keras.layers.Rescaling(1./255)
])

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(1),
])

In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    '/content/sample_data/test-task1',
    labels=Labels,
    label_mode='binary',
    class_names=None,
    color_mode='rgb',
    batch_size=128,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)


In [ ]:
history=None
strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
with strategy.scope():
  # build and compile your Keras model here
  model = tf.keras.Sequential([
    resize_and_rescale,
    data_augmentation,
    tf.keras.layers.Conv2D(
        32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Conv2D(
        64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(rate=0.5),

    tf.keras.layers.Conv2D(
        128, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Conv2D(
        256, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(
        512, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation=None)
  ])
  model.compute_output_shape(input_shape=(None, 64, 64, 3))
  model.build(input_shape=(None, 64, 64, 3))
  model.compile(optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)
  
  filepath="checkpoints"
  checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
  es = EarlyStopping(monitor='val_accuracy', patience=5)
  callbacks_list = [checkpoint, es]
  # Fit the model
  
  history = model.fit(train_ds, validation_data=val_ds, callbacks=[checkpoint, es],
    epochs=1000)
  model.save('/content/model')

In [ ]:
hist = history.history
x_arr = np.arange(len(hist['loss'])) + 1
fig = plt.figure(figsize=(12, 4))
ax = fig.add_subplot(1, 2, 1)
ax.plot(x_arr, hist['loss'], '-o', label='Train loss')
ax.plot(x_arr, hist['val_loss'], '--<', label='Validation loss')
ax.legend(fontsize=15)
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Loss', size=15)
ax = fig.add_subplot(1, 2, 2)
ax.plot(x_arr, hist['accuracy'], '-o', label='Train acc.')
ax.plot(x_arr, hist['val_accuracy'], '--<', label='Validation acc.')
ax.legend(fontsize=15)
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Accuracy', size=15)
plt.show()

TESTING

In [ ]:
model.evaluate(test_ds)

In [ ]:
model.predict(test_ds.take(1))

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Deepfake/Models/Model_1/model/')
model.summary()

In [ ]:
yhat_probs = model.predict(test_ds, verbose=0)
yhat_classes = model.predict_classes(test_ds, verbose=0)

In [ ]:
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]

In [ ]:
print(yhat_probs)

MODEL METRICS

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [ ]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_ds, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_ds, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_ds, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(test_ds, yhat_classes)
print('F1 score: %f' % f1)

In [ ]:
kappa = cohen_kappa_score(test_ds, yhat_classes)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(test_ds, yhat_probs)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(test_ds, yhat_classes)
print(matrix)

In [ ]:
loss, accuracy, f1_score, precision, recall = model.evaluate(test_ds, verbose=0)

In [ ]:
print(loss, accuracy, f1_score, precision, recall)